## [Excel Diff Reference](https://matthewkudija.com/blog/2018/07/21/excel-diff/)

In [2]:
# TODO KETE SETSUNASA NI WA
import pandas as pd
# TODO have input prompt instead of hard-coding
df_OLD = pd.read_excel("files/monthly_production_05-2019.xlsx")
#df_NEW = pd.read_excel("files/mothpro2019.xlsx")

#dfRevTest = pd.read_excel('files/monthly_revenue_04-2019.xlsx').fillna(0)
dfCYPro = pd.read_excel('files/federal_production_CY08-17.xlsx')

## For Setting Header Format

In [11]:
# Reads Header Config File
def read_hconfig():
    columns = []
    with open('config/headerdef.txt') as hdef:
        # Seperated by commas, no whitespace plz :(
        columns = hdef.readline().split(',')
    return columns

# Returns Header List based on Excel file
# Use if large amount of Field Names
def set_header(samplefile):
    return list(samplefile.columns)


header = set_header(df_OLD)
print(header)

['Month', 'Calendar Year', 'Land Class', 'Land Category', 'Commodity', 'Volume']


## For Setting Unit Dictionary

In [21]:
# Reads Unit Config File
# Commodity and Unit seperated by an equals sign " = "
# Fast, Best for small number of Units / after initial read
def read_uconfig():
    units = {}
    with open('config/unitdef.txt') as udef:
        for line in udef:
            split = line.split(" = ")
            units[split[0]] = split[1].rstrip()
    return units


# Returns Unit Dictionary on Excel file
# Very Slow, but better than manually setting large # of units
def set_units(file):
    units = {}
    for row in file['Product']:
        # For general purpose commodities
        if "(" in row:
            split = row.rsplit(" (",1)
            units[split[0]] = split[1].rstrip(")")
        # The comma is for Geothermal
        elif "," in row:
            split = row.split(", ")
            if split[0] in units:
                units[split[0]].add(split[1])
            else:
                units[split[0]] = {split[1]}
        # In case no unit is found
        else:
            units[row] = ""
            
    return units


# Wrties to config to speed up process later
# Maybe have it write seperate files for each type?
def write_units(units):
    config = open("config/unitdef.txt","w")
    for k in units:
        line = k + ' = ' + str(units[k]) + '\n'
        config.write(line)
        
        
set_units(dfCYPro)
units = read_uconfig()

## Check Header Function

In [20]:
def check_header(file):
    columns = file.columns
    uncheckedCols = set(columns)
    
    for i in range(len(header)):
        # Checks if Field in file and in correct column
        if columns.contains(header[i]):
            if columns[i] == header[i]:
                print(header[i] + ": True")
            else:
                print(header[i] + ": Wrong order")
            uncheckedCols.remove(header[i])
        else:
            # Field not present in the file
            print(header[i] + ': N/A')
    # Prints all fields not in the format
    print('\nNew Cols:', uncheckedCols)
        
check_header(dfCYPro)

Month: N/A
Calendar Year: Wrong order
Land Class: N/A
Land Category: N/A
Commodity: N/A
Volume: N/A

New Cols: {'Onshore/Offshore', 'Production Volume', 'Product', 'State/Offshore Region', 'CPS/Planning Area', 'FIPS Code', 'Category'}


## Two  Potential Unit Approaches
Compare two Dictionaries
* Harder to implement
* More modular
* Better for documentation purposes

OR Compare two Sets
* Easier to implement
* Not as flexible

In [22]:
def make_unit_set(file):
    units = set()
    for u in file['Product']:
        units.add(u)
    return units